# House, M.C.
### Generation of random text that sounds like Dr. House

In [1]:
# Packages
import urllib.request  # for grabbing the text off the internet
import os     # to help save the files of text
import re     # for replacement via regex (which I've never been good with TBH)
import nltk   # for splitting text into words
import random # for picking random words

In [2]:
# Create a directory to store episode transcripts (if it doesn't already exist)
try:
    os.mkdir('texts')
except FileExistsError:
    pass

# Get a list of transcript filenames
with open("filenames.txt", "r") as namesfile:
    filenames = namesfile.readlines()
filenames = [name.strip('\n') for name in filenames]

# Store the episode transcripts locally (if they don't already exist)
# WARNING -- There are 177 files, totaling 36.2 MB.
for name in filenames:
    if not (os.path.isfile('texts/' + name)):
        urllib.request.urlretrieve(
            'http://community.livejournal.com/clinic_duty/' + name, 
            'texts/' + name
        )

In [3]:
# Characters that will need to be removed or altered
replacements = {
    "&rsquo;" : "'",
    "&#39;" : "",
    "&hellip;" : "...",
    "\\" : "",
    '“' : '',
    '”' : '',
    "’" : "'",
    '"' : '',
    '&lt;' : '',
    " -- " : " — ",
    " - " : " — "
}

# Extract a list of the complete text spoken by Dr. House in every episode
episodes = []
for name in filenames:
    with open('texts/' + name, 'r', encoding="utf8") as myfile:
        data = myfile.read().replace('\n', '').split('<br />')
        data = ' '.join([
            re.sub( # eliminate any [text in brackets]
                    r'\[(.*?)\]|\<(.*?)\>|\((.*?)\)', '', line
                ).replace(  # The transcripts are very nonstandardized
                    'GREG HOUSE: ', ''
                ).replace(  # in the sense that there are four different
                    'House: ', ''
                ).replace(  # ways that House's lines may be marked
                    'HOUSE: ', ''
                ).replace(  # so we have to deal with each one :(
                    'House : ', ''
                )  
            for line in data 
            if line.lower().startswith('house:') 
                or line.lower().startswith('greg house:')
                or line.lower().startswith('house :')
        ])
        for key in replacements:
            data = data.replace(key, replacements[key])
        episodes.append(data)

# Join all episodes into one large text string
alltext = ' '.join(ep for ep in episodes)

In [4]:
# Build the list of words used
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+|[^\w\s]+")
stream = tokenizer.tokenize(alltext)

# Build a list of word pairs used (AKA 2-grams)
stream2 = [(stream[i], stream[i+1]) for i in range(len(stream)-1)]

# A function that takes a word and randomly chooses a word to follow it
def nextword(word):
    return stream[
        random.choice([
                i for i, j in enumerate(stream) 
                if (j == word) & (i < len(stream)-1)
            ]) + 1
    ]

# A function that takes a pair of words and 
# randomly chooses a word to follow them
def nextpair(pair):
    return stream2[
        random.choice([
                i for i, j in enumerate(stream2) 
                if (j == pair) & (i < len(stream2)-2)
            ]) + 1
    ]

# To clean up the generated text
replacements = {
    ' .' : '.',
    ' ,' : ',',
    " ' " : "'",
    " ?" : "?",
    " !" : "!",
    " ’ " : "’",
    " ; " : "; ",
    " ‘ " : " ‘",
    "$ " : "$",
    " …" : "...",
    " - " : "-"
}
def postprocess(text):
    for key in replacements:
        text = text.replace(key, replacements[key])
    return text

In [5]:
# Generate text using single words (not very good)
words = [random.choice([word for word in stream if word[0].isupper()])]
while (
    (words[-1] != '.') & (words[-1] != '!') & (words[-1] != '?')
) | (len(words) < 10):
    words.append(nextword(words[-1]))
postprocess(' '.join(words))

"I've got about her in the other can get it could use toe, I... I thought your blood."

In [6]:
# Generate text using 2-grams:
pair = random.choice([pair for pair in stream2 if pair[0][0].isupper()])
words = [pair[0], pair[1]]
while (
    (words[-1] != '.') & (words[-1] != '!') & (words[-1] != '?')
) | (len(words) < 10):
    pair = nextpair(pair)
    words.append(pair[-1])
postprocess(' '.join(words))

'Leukoencephalopathy? What does it matter what it looks like.'